# Sentiment Analysis on The Reddit Climate Change Dataset
by Santiago Segovia

### Motivation

We will use the Reddit Climate Change Dataset to fine-tune a DistilBERT model that is able to classify whether a comment in Reddit towards climate change is positive or negative. The analysis follows [this publication](https://huggingface.co/blog/sentiment-analysis-python) from Hugginface.

The first pre-trained model we use is `distilbert-base-uncased`. We'll use this results as a baseline

### 1. Install Dependencies and Initial Setup

In [1]:
!pip install datasets transformers accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch

from google.colab import drive
from transformers import pipeline

In [4]:
USE_CUDA = torch.cuda.is_available()

if USE_CUDA:
    DEVICE = torch.device('cuda')
    train_obs = 10000
    test_obs = 1000
    print("Using cuda.")
else:
    DEVICE = torch.device('cpu')
    train_obs = 3000
    test_obs = 300
    print("Using cpu.")

Using cpu.


In [5]:
# Mount GDrive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
# Load data (takes ~1 min to load)
import csv

small_data = True
dtype_dict = {'label': int}
data_path = "/content/drive/Shareddrives/adv-ml-project/Data/"
comments = pd.read_csv(data_path + "by_threshold/comments_filtered.csv", quoting=csv.QUOTE_NONNUMERIC, dtype=dtype_dict)

### 2. Preprocess Data

In order to evaluate the performance of our model, we need to create a train-test split. We randomly pick 20% of the records and identify them as part of the testing dataset:

In [8]:
import random
random.seed(120938)

num_test_obs = round(comments.shape[0] * 0.2)
ids_test_obs = random.sample(range(comments.shape[0]), num_test_obs)

In [9]:
comments['test_split'] = 0
comments.loc[ids_test_obs,'test_split'] = 1

In [ ]:
comments.head(3)

,subreddit.name,date,body,sentiment,label,test_split
0,askreddit,2022-08-31 23:56:46,I think climate change tends to get some peopl...,0.6634,1,0
1,askreddit,2022-08-31 23:54:25,They need to change laws so it's more worth se...,0.4690,1,1
2,askreddit,2022-08-31 23:52:41,That a big part of the solution to climate cha...,0.8937,1,0


We need to convert our data to an iterable dataset to easily use Hugginface's functions. We use the `from_dict()` method from the `datasets` [module](https://huggingface.co/docs/datasets/en/create_dataset).

In [10]:
# Fill NaN values with empty strings, otherwise from_dict will raise an error
comments['body'] = comments['body'].fillna('')

In [11]:
from datasets import Dataset

def train_test_split(df):
  train_dict = {"text": df.loc[df['test_split'] == 0, 'body'].tolist(),
                "label": df.loc[df['test_split'] == 0, 'label'].tolist()}
  test_dict = {"text": df.loc[df['test_split'] == 1, 'body'].tolist(),
               "label": df.loc[df['test_split'] == 1, 'label'].tolist()}
  train_df = Dataset.from_dict(train_dict)
  test_df = Dataset.from_dict(test_dict)

  return train_df, test_df

# Create train and test data
train_data, test_data = train_test_split(comments)

In [12]:
# Create a smaller training dataset for faster training times
small_train_dataset = train_data.shuffle(seed=42).select([i for i in list(range(train_obs))])
small_test_dataset = test_data.shuffle(seed=42).select([i for i in list(range(test_obs))])

In [13]:
# Example of structures
print(small_train_dataset[0])
print(small_test_dataset[0])

{'text': "&gt;prove how the Democratic Party establishment is a nightmare.\n\n* No action on climate change.\n* No action on wealth inequality\n* Failed health insurance legislation\n* Lost to Trump\n* Lost 1000 elected positions in 10 years\n* Supported Bush's illegal Iraq war\n* Support obscenely bloated defense budget\n* Interfere in primary elections\n* Don't represent their constituents, only represent billionaires\n* Love them some Wall Street\n* The list is endless", 'label': 0}
{'text': "I've also noticed our weather patterns change. Used to be that what happened in Vancouver was happening 100km inland, north and south. Now there are small weather pockets all the time. I've driven through Metro Vancouver and gone through hail, rain, snow, and warm sun in less than 20 minutes. Never happened as a kid.\n\nTo me, those types of changes, especially when it's every year and all the time, are great examples of how weather's affected by and a symptom of climate change.", 'label': 1}


To [preprocess](https://huggingface.co/docs/transformers/preprocessing#everything-you-always-wanted-to-know-about-padding-and-truncation) our data, we will use [DistilBERT tokenizer](https://huggingface.co/docs/transformers/v4.15.0/en/model_doc/distilbert#transformers.DistilBertTokenizer):


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Next, we prepare the text inputs for the model for both splits of our dataset (training and test) by using the [map method](https://huggingface.co/docs/datasets/en/process#map) with [batch processing](https://huggingface.co/docs/datasets/en/process#batch-processing):

In [ ]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

if small_data:
    tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
    tokenized_test = small_test_dataset.map(preprocess_function, batched=True)
else:
    tokenized_train = train_data.map(preprocess_function, batched=True)
    tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Sentences aren’t always the same length which can be an issue because the model inputs need to have a uniform shape. Padding is a strategy for ensuring tensors are rectangular by adding a special padding token to shorter sentences. We use a [Data Collator](https://huggingface.co/docs/transformers/en/main_classes/data_collator) to convert our training samples to PyTorch tensors and concatenate them with the correct amount of [padding](https://huggingface.co/docs/transformers/preprocessing#pad):

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

### 3. Training the model

For training, we will be using the [Trainer API](https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.Trainer), which is optimized for fine-tuning Transformers models such as DistilBERT.

First, we define DistilBERT as our base model:

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Then, we define the metrics we'll be using to evaluate how good is the fine-tuned model ([accuracy and f1 score](https://huggingface.co/metrics)):

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

Before training our model, we need to define the training arguments and define a Trainer with all the objects we constructed up to this point:

In [ ]:
from transformers import TrainingArguments, Trainer

output_name = data_path + "Output"

training_args = TrainingArguments(
   output_dir=output_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

Now, we train the model on the dataset:

In [ ]:
trainer.train()

Step,Training Loss
500,0.507100
1000,0.339100


TrainOutput(global_step=1250, training_loss=0.40099765625, metrics={'train_runtime': 771.1868, 'train_samples_per_second': 25.934, 'train_steps_per_second': 1.621, 'total_flos': 2152081916990208.0, 'train_loss': 0.40099765625, 'epoch': 2.0})

Finally, we compute the evaluation metrics to see how good your model is:

In [ ]:
trainer.evaluate()

<ipython-input-17-dcbaed1ca091>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 0.41115686297416687,
 'eval_accuracy': 0.824,
 'eval_f1': 0.8181818181818182,
 'eval_runtime': 16.1086,
 'eval_samples_per_second': 62.079,
 'eval_steps_per_second': 3.911,
 'epoch': 2.0}